# Analysis for Supply Chain Optimisation and Pricing Strategy
Exploration of Online Retail Dataset from UCI [linked here](https://archive.ics.uci.edu/dataset/352/online+retail), to learn the basics of analytics in SCM and segment the product assortment based on profitability or value contribution to the business.

**Objective:** Perform an ABC Analysis using historical sales data from the dataset.

**Actions:** Clearly identify Category A (high-value/high-profit) items that must always be in stock, and Category C (low-value/slow-moving) items that could be considered for clearance or removal from the inventory, thereby optimising warehouse space and reducing capital tied up in stock.

## Download the dataset from the URL
We use the handy util function `download_raw_dataset` to download it straight from UCI to keep it easy to reproduce.

In [1]:
# Imports for this section
from utils.data import download_raw_dataset

In [2]:
download_raw_dataset()

Dataset seems to already exist, please delete manually or use force option to redownload.


## Load the data into a Pandas DataFrame

In [3]:
# Imports for this section
import os

import pandas as pd

In [4]:
dataset_file = os.path.join(os.getcwd(), "data", "Online Retail.xlsx")

In [5]:
data = pd.read_excel(dataset_file)

## Exploration
The `head` and `describe` utility is a quick way to check the data structure and any issues with the dataset before diving deep.

In [6]:
data.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [7]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


As we can see the data is not without it's fair share of problems. Quantity has negative values and so does UnitPrice (check out the original link for meaning of the headers). We can also see that customer id is missing for some of the transactions.

In [8]:
# Check how many bad records (for quantity, unitprice and customer id)
print(f"{data[data['Quantity'] < 0]['Quantity'].count()} records have negative quantity")
print(f"{data[data['UnitPrice'] < 0]['UnitPrice'].count()} records have negative price")
print(f"{data['CustomerID'].isna().sum()} records have no customer ids")
print(f"{data['InvoiceNo'].astype(str).str.startswith('C', na=False).sum()} records are cancellations")

10624 records have negative quantity
2 records have negative price
135080 records have no customer ids
9288 records are cancellations


These given an idea of how much data has to be cleaned/dropped to prevent it from affecting the accuracy of our analysis. The next section dives into it in detail.

## More Exploration Deep Dive and Cleaning

In [9]:
# Get the InvoiceId for negative quantity and unit price and see if there's any information there.
bad_quant_invoices = data[data['Quantity'] < 0]['InvoiceNo'].unique()
bad_quant_invoices

array(['C536379', 'C536383', 'C536391', ..., 'C581499', 'C581568',
       'C581569'], shape=(5172,), dtype=object)

Invoices starting with a **'c'** indicates that it is a cancelled order. We will check if this is the case throughout.

In [10]:
print(len(bad_quant_invoices))
print(len([i for i in list(bad_quant_invoices) if str(i)[0] == "C"]))

5172
3836


In [11]:
bad_price_invoices = data[data['UnitPrice'] < 0]['InvoiceNo'].unique()
bad_price_invoices

array(['A563186', 'A563187'], dtype=object)

There's no relationship between bad quantity, prices with invoice id, we will clean them separately.

In [12]:
# Create a copy so that the cleaning is reproducible
cleaned_data = data.copy()

# Now we can just drop all cancellations first, since they do not contribute to the sales needed for our ABC analysis
cleaned_data = cleaned_data[~cleaned_data['InvoiceNo'].astype(str).str.startswith('C', na=False)]
# Drop all the negative quantity
cleaned_data = cleaned_data[cleaned_data['Quantity'] > 0]
# Drop all the negative UnitPrice and (zero since they do not contribute to profits)
cleaned_data = cleaned_data[cleaned_data['UnitPrice'] > 0]
# Best to drop no customer ids, as we do not know if these are actual sales?
cleaned_data = cleaned_data[~cleaned_data['CustomerID'].isna()]

print(f"Final number of records that can be used for analysis: {cleaned_data.shape[0]}")

Final number of records that can be used for analysis: 397884


## Feature Engineering and Aggregation

In [25]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [20]:
# Rename the InvoiceDate as Date
cleaned_data['Date'] = cleaned_data['InvoiceDate'].dt.date
# Revenue - The actual sales, since we don't have profit margin
# This is our ABC feature.
cleaned_data['Revenue'] = cleaned_data['Quantity'] * cleaned_data['UnitPrice']

Since we do not have profit margins for each items, we have to calculate the ABC from just the revenue, which is not the most accurate but works for learning purposes.

In [14]:
# 1. Aggregate to the product level
product_df = cleaned_data.groupby('StockCode').agg(
    Description=('Description', 'first'),
    Total_Revenue=('Revenue', 'sum'),
    Total_Quantity=('Quantity', 'sum')
).reset_index()

# 2. Sort by Revenue (descending)
product_df = product_df.sort_values(by='Total_Revenue', ascending=False)

# 3. Calculate cumulative revenue
product_df['Cumulative_Revenue'] = product_df['Total_Revenue'].cumsum()

# 4. Calculate percentage of total revenue
total_revenue_all = product_df['Total_Revenue'].sum()
product_df['Revenue_Percentage'] = (product_df['Total_Revenue'] / total_revenue_all) * 100
product_df['Cumulative_Revenue_Percentage'] = (product_df['Cumulative_Revenue'] / total_revenue_all) * 100

In [15]:
product_df

,StockCode,Description,Total_Revenue,Total_Quantity,Cumulative_Revenue,Revenue_Percentage,Cumulative_Revenue_Percentage
2329,23843,"PAPER CRAFT , LITTLE BIRDIE",168469.600,80995,168469.600,1.890494e+00,1.890494
1218,22423,REGENCY CAKESTAND 3 TIER,142592.950,12402,311062.550,1.600117e+00,3.490611
3349,85123A,WHITE HANGING HEART T-LIGHT HOLDER,100603.500,36782,411666.050,1.128929e+00,4.619540
3343,85099B,JUMBO BAG RED RETROSPOT,85220.780,46181,496886.830,9.563111e-01,5.575851
1927,23166,MEDIUM CERAMIC TOP STORAGE JAR,81416.730,77916,578303.560,9.136236e-01,6.489475
...,...,...,...,...,...,...,...
2747,90104,PURPLE FRANGIPANI HAIRCLIP,0.850,1,8911405.791,9.538336e-06,99.999976
2733,90084,PINK CRYSTAL GUITAR PHONE CHARM,0.850,1,8911406.641,9.538336e-06,99.999986
390,21268,VINTAGE BLUE TINSEL REEL,0.840,2,8911407.481,9.426120e-06,99.999995
2505,84227,HEN HOUSE W CHICK IN NEST,0.420,1,8911407.901,4.713060e-06,100.000000


This cumulative revenue and percentage tells us which products have the most sales as a running sum. We can now assign the products as A, B and C, since this is sorted by Revenue in descending order!

In [16]:
# 5. Assign ABC Categories
def assign_abc(pct):
    if pct <= 80:
        return 'A (High Value)'
    elif pct <= 95:
        return 'B (Medium Value)'
    else:
        return 'C (Low Value)'

product_df['ABC_Category'] = product_df['Cumulative_Revenue_Percentage'].apply(assign_abc)

In [17]:
product_df

,StockCode,Description,Total_Revenue,Total_Quantity,Cumulative_Revenue,Revenue_Percentage,Cumulative_Revenue_Percentage,ABC_Category
2329,23843,"PAPER CRAFT , LITTLE BIRDIE",168469.600,80995,168469.600,1.890494e+00,1.890494,A (High Value)
1218,22423,REGENCY CAKESTAND 3 TIER,142592.950,12402,311062.550,1.600117e+00,3.490611,A (High Value)
3349,85123A,WHITE HANGING HEART T-LIGHT HOLDER,100603.500,36782,411666.050,1.128929e+00,4.619540,A (High Value)
3343,85099B,JUMBO BAG RED RETROSPOT,85220.780,46181,496886.830,9.563111e-01,5.575851,A (High Value)
1927,23166,MEDIUM CERAMIC TOP STORAGE JAR,81416.730,77916,578303.560,9.136236e-01,6.489475,A (High Value)
...,...,...,...,...,...,...,...,...
2747,90104,PURPLE FRANGIPANI HAIRCLIP,0.850,1,8911405.791,9.538336e-06,99.999976,C (Low Value)
2733,90084,PINK CRYSTAL GUITAR PHONE CHARM,0.850,1,8911406.641,9.538336e-06,99.999986,C (Low Value)
390,21268,VINTAGE BLUE TINSEL REEL,0.840,2,8911407.481,9.426120e-06,99.999995,C (Low Value)
2505,84227,HEN HOUSE W CHICK IN NEST,0.420,1,8911407.901,4.713060e-06,100.000000,C (Low Value)


In [18]:
# Show the top-performing products and their category
print("--- Top 5 Products (Category A) ---")
print(product_df.head(5))

# Summarize the count of products in each category
abc_summary = product_df.groupby('ABC_Category')['StockCode'].count().reset_index()
abc_summary.rename(columns={'StockCode': 'Product_Count'}, inplace=True)
print("\n--- ABC Category Summary ---")
print(abc_summary)

--- Top 5 Products (Category A) ---
     StockCode                         Description  Total_Revenue  \
2329     23843         PAPER CRAFT , LITTLE BIRDIE      168469.60   
1218     22423            REGENCY CAKESTAND 3 TIER      142592.95   
3349    85123A  WHITE HANGING HEART T-LIGHT HOLDER      100603.50   
3343    85099B             JUMBO BAG RED RETROSPOT       85220.78   
1927     23166      MEDIUM CERAMIC TOP STORAGE JAR       81416.73   

      Total_Quantity  Cumulative_Revenue  Revenue_Percentage  \
2329           80995           168469.60            1.890494   
1218           12402           311062.55            1.600117   
3349           36782           411666.05            1.128929   
3343           46181           496886.83            0.956311   
1927           77916           578303.56            0.913624   

      Cumulative_Revenue_Percentage    ABC_Category  
2329                       1.890494  A (High Value)  
1218                       3.490611  A (High Value)  
33

As we can see **776** of 3665 products make up for 80% of all generated revenue. The next 922 make the next 15% and the remaining 1967 products only make the remaining 5%.

## Pricing Strategy
Now that we have identified the ABC products, we already have enough information to know on which products to focus and what to clear inventory on. However, for better profits, we can see if we can adjust the pricing of our best sellers for them to sell more for even more profits. We can focus on our best product for example.

In [31]:
# Select the top product from Category A
focus_stock_code = product_df.iloc[1]['StockCode']
print(f"\n--- Focusing on Top Seller: {focus_stock_code} ---")

# Aggregate the cleaned data to the daily level for the focus product
elasticity_df = cleaned_data[cleaned_data['StockCode'] == focus_stock_code].groupby('Date').agg(
    # Quantity is the Y (Demand) variable
    Daily_Quantity=('Quantity', 'sum'),
    # UnitPrice average is the main X (Price) variable
    Avg_Price=('UnitPrice', 'mean'),
    # Use the existing date features for controls
    InvoiceDate=('InvoiceDate', 'first')
).reset_index()

# Re-extract time features onto the new aggregated DataFrame
elasticity_df['DayOfWeek'] = elasticity_df['InvoiceDate'].dt.dayofweek
elasticity_df['Month'] = elasticity_df['InvoiceDate'].dt.month


--- Focusing on Top Seller: 22423 ---


In [32]:
elasticity_df

,Date,Daily_Quantity,Avg_Price,InvoiceDate,DayOfWeek,Month
0,2010-12-01,28,12.525000,2010-12-01 12:27:00,2,12
1,2010-12-02,184,12.422727,2010-12-02 09:29:00,3,12
2,2010-12-03,152,12.235714,2010-12-03 10:26:00,4,12
3,2010-12-05,92,12.210000,2010-12-05 10:12:00,6,12
4,2010-12-06,43,12.750000,2010-12-06 11:42:00,0,12
...,...,...,...,...,...,...
295,2011-12-05,33,13.906667,2011-12-05 08:49:00,0,12
296,2011-12-06,146,12.492857,2011-12-06 10:00:00,1,12
297,2011-12-07,130,11.978571,2011-12-07 09:02:00,2,12
298,2011-12-08,16,12.750000,2011-12-08 10:33:00,3,12


In [33]:
# Transform variables to log-scale for direct elasticity interpretation
elasticity_df['Log_Quantity'] = np.log(elasticity_df['Daily_Quantity'])
elasticity_df['Log_Price'] = np.log(elasticity_df['Avg_Price'])

print(f"\nElasticity Data Prepared for {focus_stock_code}. Total data points: {elasticity_df.shape[0]}")
print(elasticity_df.head())


Elasticity Data Prepared for 22423. Total data points: 300
         Date  Daily_Quantity  Avg_Price         InvoiceDate  DayOfWeek  \
0  2010-12-01              28  12.525000 2010-12-01 12:27:00          2   
1  2010-12-02             184  12.422727 2010-12-02 09:29:00          3   
2  2010-12-03             152  12.235714 2010-12-03 10:26:00          4   
3  2010-12-05              92  12.210000 2010-12-05 10:12:00          6   
4  2010-12-06              43  12.750000 2010-12-06 11:42:00          0   

   Month  Log_Quantity  Log_Price  
0     12      3.332205   2.527727  
1     12      5.214936   2.519528  
2     12      5.023881   2.504359  
3     12      4.521789   2.502255  
4     12      3.761200   2.545531  


In [34]:
# Y: Log Quantity (Demand)
Y = elasticity_df['Log_Quantity']

# X: Log Price (main predictor) + Controls (DayOfWeek, Month)
X = elasticity_df[['Log_Price', 'DayOfWeek', 'Month']]

# Add a constant term required for the statsmodels library
X = sm.add_constant(X)

# --- 2. Split Data (Good Practice) ---
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# --- 3. Build and Fit the Model ---
model = sm.OLS(Y_train, X_train).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           Log_Quantity   R-squared:                       0.201
Model:                            OLS   Adj. R-squared:                  0.191
Method:                 Least Squares   F-statistic:                     19.79
Date:                Mon, 22 Sep 2025   Prob (F-statistic):           1.77e-11
Time:                        16:07:13   Log-Likelihood:                -317.96
No. Observations:                 240   AIC:                             643.9
Df Residuals:                     236   BIC:                             657.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         28.1371      3.292      8.547      0.0

In [35]:
elasticity_coefficient = model.params['Log_Price']
print(f"\nPrice Elasticity Coefficient (η) for Top Product: {elasticity_coefficient:.3f}")


Price Elasticity Coefficient (η) for Top Product: -9.854


## Final Deliverables: Supply Chain & Pricing Strategy Insights

Based on the data cleaning, feature engineering, **ABC Analysis**, and **OLS Regression model results**, here are the key conclusions and actionable recommendations for **Supply Chain** and **Pricing Strategy**.

---

### 1. Inventory Optimisation: ABC Analysis (Supply Chain)

The **ABC (Pareto) Analysis** was performed using **Total Revenue** as the primary value metric, identifying products that drive the majority of business value.

#### Key Finding: The Pareto Principle in Action

The analysis confirms the Pareto Principle, where a small percentage of products account for a disproportionately large share of total revenue.

- **Total Revenue Analyzed:** £8,911,407.90  
- **Total Unique Products:** 3,665  
- **Category A:** Products contributing up to 80% of the cumulative total revenue.

**Top Revenue Product:**  
The single largest contributor to revenue is **PAPER CRAFT, LITTLE BIRDIE** (StockCode: 23843), generating **£168,469.60** in sales alone.

#### Recommendation

| Category | Recommended Strategy       | Rationale                                                                 |
|---------|--------------------------|---------------------------------------------------------------------------|
| **A (High Value)** | Strict Inventory Control | These items are critical to revenue stability. Implement zero-tolerance for stock-outs, use precise forecasting, and apply the tightest security and control measures. |
| **C (Low Value)** | Review or Clearance       | These items account for minimal revenue but occupy physical space. Review for clearance, discontinuing, or bundling to free up capital and warehouse space. |

---

### 2. Pricing Strategy: Price Elasticity Model

A **Multiple Linear Regression model** was built using **log-transformed demand (Log Quantity)** and **log-transformed price (Log Price)** for a frequently sold Category A product to determine its **Price Elasticity of Demand (η)**.

#### Key Finding: Highly Elastic Demand

- **Price Elasticity Coefficient (η):** -9.854  
- **Statistical Significance:** p-value = 0.000 (highly significant)  

**Interpretation:**  
Since the absolute value of η (9.854) is much greater than 1, the product is **highly elastic**.  
A **1% increase in price** is predicted to cause a **9.854% decrease in quantity demanded**.

#### Recommendation

| Current State | Recommendation                    | Business Impact |
|---------------|----------------------------------|----------------|
| Elasticity (∣η∣ > 1) | Lower the Price or Run Aggressive Promotions | A price reduction will be met with a disproportionately large surge in demand, leading to a significant overall increase in total revenue and market share for this key product. |

---

#### Model Diagnostics

- **R-squared:** 0.201 → Model explains 20.1% of the variance in demand (reasonable starting point for retail demand modeling).
- **DayOfWeek:** Statistically significant → Suggests minor but measurable weekly seasonality effects on demand.

---
